# Kinetic temperature

The kinetic temperature is obtained with the 12CO amplitude :
$$T_{kin}=T_{ex}=\frac{5.53}{\ln\left(1+\frac{5.53}{T_R+0.148}\right)}$$

In [3]:
from src.hdu.maps.map import Map
from src.hdu.cubes.cube_co import CubeCO
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_kinetic_temperature

In [2]:
GroupedMaps([(
    "kinetic_temperature", [
        get_kinetic_temperature(amp) for amp in Tesseract.load(
            "data/Loop4_co/N1/12co/object_filtered.fits"
        ).to_grouped_maps().amplitude
    ]
)]).save("t.fits")

File overwritten.


# Column density

The column density is obtained with
\begin{align*}
    N(^{13}\mathrm{CO})=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{Qh\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)
\end{align*}
Knowing that
$$Q=\frac{2T_{ex}^{13}}{h\nu_{13}/k}$$
we can simplify to
\begin{align*}
    N(^{13}\mathrm{CO})&=\frac{3h\Delta V_{13}}{8\pi^3\mu^2}\left(\frac{\frac{2T_{ex}^{13}}{h\nu_{13}/k}h\nu_{13}/k}{1-e^{-\frac{h\nu_{13}}{kT_{ex}}}}\right)\\
    &=\frac{3h\Delta V_{13}T_{ex}^{13}}{4\pi^3\mu^2\left(1-e^{-\frac{h\nu_{13}}{kT_{ex}}}\right)}\\
\end{align*}

In [106]:
import numpy as np
import graphinglib as gl

from src.hdu.maps.convenient_funcs import get_13co_column_density

In [108]:
cube_12co = CubeCO.load("data/Loop4_co/N1/12co/Loop4N1_wcs.fits")
cube_13co = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")
maps_12co = Tesseract.load("data/Loop4_co/N1/12co/object_filtered.fits").to_grouped_maps()
maps_13co = Tesseract.load("data/Loop4_co/N1/13co/tesseract_corrected.fits").to_grouped_maps()

# The right gaussians first need to be selected
# This solution is for single component 13co maps
assert len(maps_13co.mean) == 1
mean_12co = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.mean], axis=0)
diff_array= np.abs(mean_12co - maps_13co.mean[0].data)  # Compute the difference between the centroid of every gaussian
nan_mask = np.isnan(diff_array)     # Apply a nan mask to allow proper argmin use
diff_array[nan_mask] = 2**7-1       # Remove nans
min_mask = np.argmin(diff_array, axis=0)
get_gaussian = lambda arr: np.take_along_axis(arr, min_mask[np.newaxis, ...], axis=0).squeeze()
amp_12co_val = np.stack([m.get_reprojection_on(maps_13co.mean[0]).data for m in maps_12co.amplitude], axis=0)
amp_12co_unc = np.stack([m.get_reprojection_on(maps_13co.mean[0]).uncertainties for m in maps_12co.amplitude], axis=0)

get_13co_column_density(
    fwhm_13co=maps_13co.stddev[0]*2*np.sqrt(2*np.log(2)),
    kinetic_temperature_13co=get_kinetic_temperature(maps_13co.amplitude[0]/0.43),
    kinetic_temperature_12co=Map(get_gaussian(amp_12co_val), get_gaussian(amp_12co_unc))
).save("tt.fits")

# result = np.take_along_axis(mean_12co, min_mask[np.newaxis, ...], axis=0).squeeze()
# fig = gl.Figure()
# fig.add_elements(gl.Heatmap(result, origin_position="lower", color_map="viridis"))
# fig.show()
